In [6]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version
# from pyspark import SparkContext, SparkConf

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

# conf = SparkConf().set('spark.executor.extraJavaOptions','-Dcom.amazonaws.services.s3.enableV4=true'). \
#  set('spark.driver.extraJavaOptions','-Dcom.amazonaws.services.s3.enableV4=true'). \
#  setAppName('pyspark_aws').setMaster('local[*]')
# sc=SparkContext(conf=conf)
# sc.setSystemProperty('com.amazonaws.services.s3.enableV4', 'true')


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists

In [7]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-15 16:32:42--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.48MB/s    in 0.2s    

2023-03-15 16:32:42 (4.48 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [8]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [9]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_download_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
video_download_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12190288|R3FU16928EP5TC|B00AYB1482|     668895143|Enlightened: Seas...|Digital_Video_Dow...|          5|            0|          0|   N|                Y|I loved it and I ...|I loved it and I ...| 2015-08-31|
|         US|   30549954|R1IZHHS1MH3AQ4|B00KQD28OM|     246219280|             Vicious|Digital_Video_Dow

In [10]:
# Get the number of rows in the DataFrame.
video_download_df.count()

4057147

# Transform the Data

## Create the "review_id_table".

In [11]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df=video_download_df.select("review_id", "customer_id", "product_id", "product_parent", "review_date")
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3FU16928EP5TC|   12190288|B00AYB1482|     668895143| 2015-08-31|
|R1IZHHS1MH3AQ4|   30549954|B00KQD28OM|     246219280| 2015-08-31|
| R52R85WC6TIAH|   52895410|B01489L5LQ|     534732318| 2015-08-31|
| R7HOOYTVIB0DS|   27072354|B008LOVIIK|     239012694| 2015-08-31|
|R1XQ2N5CDOZGNX|   26939022|B0094LZMT0|     535858974| 2015-08-31|
|R1HCST57W334KN|    4772040|B0112OSOQE|      38517795| 2015-08-31|
|R32BUTYQS1ZJBQ|   12910040|B000NPE5SA|     373323715| 2015-08-31|
|  RH4SXPL4L9QU|   38805573|B00XWV4QXG|     633842417| 2015-08-31|
|R37INWIQA5YW8N|   37100714|B00X8UKOUK|     666093513| 2015-08-31|
|R18GSVAAS3N8GX|   41234409|B00OOKXTFU|     801680808| 2015-08-31|
|R2QCG0ZZOHYUOX|   24698954|B00URDJ600|     174981485| 2015-08-31|
|R2B1OCYMUVMS5C|   11956809|B009JUZ5I4|     654293980| 2015-08

## Create the "products" Table

In [12]:
# checking to see if their would be a lot of dups in the first 100
review_id_df.orderBy(review_id_df["product_id"])
review_id_df.show(100)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3FU16928EP5TC|   12190288|B00AYB1482|     668895143| 2015-08-31|
|R1IZHHS1MH3AQ4|   30549954|B00KQD28OM|     246219280| 2015-08-31|
| R52R85WC6TIAH|   52895410|B01489L5LQ|     534732318| 2015-08-31|
| R7HOOYTVIB0DS|   27072354|B008LOVIIK|     239012694| 2015-08-31|
|R1XQ2N5CDOZGNX|   26939022|B0094LZMT0|     535858974| 2015-08-31|
|R1HCST57W334KN|    4772040|B0112OSOQE|      38517795| 2015-08-31|
|R32BUTYQS1ZJBQ|   12910040|B000NPE5SA|     373323715| 2015-08-31|
|  RH4SXPL4L9QU|   38805573|B00XWV4QXG|     633842417| 2015-08-31|
|R37INWIQA5YW8N|   37100714|B00X8UKOUK|     666093513| 2015-08-31|
|R18GSVAAS3N8GX|   41234409|B00OOKXTFU|     801680808| 2015-08-31|
|R2QCG0ZZOHYUOX|   24698954|B00URDJ600|     174981485| 2015-08-31|
|R2B1OCYMUVMS5C|   11956809|B009JUZ5I4|     654293980| 2015-08

In [13]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = video_download_df.dropDuplicates(['product_id','product_title'])
products_df.count()

166748

## Create the "customers" Table

In [14]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
 
customers_df = review_id_df.groupBy("customer_id").count()
customers_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   36427874|    1|
|   17687096|   10|
|   43798350|    4|
|   49387236|    1|
|   24321836|    2|
|   42543176|    1|
|   16298288|    4|
|   12333316|    1|
|   25790304|    1|
|   13786319|    1|
|   24796052|    1|
|   53049917|    4|
|   30478898|    1|
|   20829560|    3|
|   29699080|    3|
|    8681732|    3|
|   34220092|    2|
|   13727857|   14|
|   13480593|    3|
|   49935991|    1|
+-----------+-----+
only showing top 20 rows



## Create the "vine_table".

In [15]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df=video_download_df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3FU16928EP5TC|          5|            0|          0|   N|
|R1IZHHS1MH3AQ4|          5|            0|          0|   N|
| R52R85WC6TIAH|          4|           17|         18|   N|
| R7HOOYTVIB0DS|          5|            0|          0|   N|
|R1XQ2N5CDOZGNX|          5|            0|          0|   N|
|R1HCST57W334KN|          5|            5|          6|   N|
|R32BUTYQS1ZJBQ|          4|            1|          1|   N|
|  RH4SXPL4L9QU|          3|            0|          0|   N|
|R37INWIQA5YW8N|          2|            0|          0|   N|
|R18GSVAAS3N8GX|          3|            0|          0|   N|
|R2QCG0ZZOHYUOX|          5|            0|          0|   N|
|R2B1OCYMUVMS5C|          5|            0|          0|   N|
|R131ZJWHK0XXY0|          4|            0|          0|   N|
| RY6RIUNL03N8M|          5|            

# Load

In [16]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "xxxxxxxxxxxx", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)


In [ ]:
# Write products_df to table in RDS
product_df.write.jdbc(url=jdbc_url, table='product_table', mode=mode, properties=config)


In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)


In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)
